# Homework 4 - Kaggle Competition
## CSCI 5622 - Spring 2019
#Scott Scheraga
***
train-features.csv contains the features of the examples / cases, separated by commas, that you will use to train your classifiers. You must associate the features found in each row / line with the output found in the corresponding row / line in train-output.csv to create a full case.

test-features.csv contains the features of the examples / cases that you will need to predict using your trained model.

See train.names for header descriptions of the train and test features. This will be helpful in understanding how you may need to normalize, bin, or otherwise update your data to better suit your classifiers.

You can use test-submission.demo to see what an example submission file should look like. This file is a baseline case - you can even upload it to make sure your Kaggle account is working correctly!


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import pandas as pd
from sklearn import svm, metrics
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split

from collections import defaultdict
import math
from random import seed
from random import random

%matplotlib inline



Batching notes:
Intial analysis of the training set showed the following information:
from print(traincombined['output'].value_counts())
        print(traincombined.groupby('output').mean())
24720 people below threshold, 7841 above.


As marital status averages were the following, I binned by married vs unmarried due to notably different outputs. 

--marital-status        |||         output                   
 Divorced                    0.104209  
 Married-AF-spouse           0.434783  
 Married-civ-spouse          0.446848  
 Married-spouse-absent       0.081340  
 Never-married               0.045961  
 Separated                   0.064390  
 Widowed                     0.085599  
 
Similarly, for relationship, I binned by Spouse vs not spouse for the same reason as above, and then decided that the relationship column was redunant to marital status, and then deleted it. 

--Relationship      |||  output                    
 Husband              0.448571  
 Not-in-family        0.103070  
 Other-relative       0.037717  
 Own-child            0.013220  
 Unmarried            0.063262  
 Wife                 0.475128 
 
There is a very notable increase in capital gain and loss for people above the income threshold. I decided on a 4 bins for capital gain and capital loss. 

--Averages for 0--
 age 36.783738 
 education-num 9.595065 
 capital-gain 148.752468
 capital-loss 53.142921 
 hours-per-week 38.840210  

--Averages for 1--
 age 44.249841
 education-num 11.611657  
 capital-gain 4006.142456  
 capital-loss 195.001530 
 hours-per-week 45.473026 




In [6]:
class DataA:
    def __init__(self):
       
        #traindatafeatures= pd.read_csv("train-features_scottannotated.csv")
        trainoutput= pd.read_csv("train-output_scottannotated.csv")
        testdatafeatures= pd.read_csv("test-features_scottannotated.csv")
        
        
        traindatafeatures= pd.read_csv("traincombined.csv")
        #traindatafeatures =traindatafeatures.dropna()
        print(traindatafeatures.groupby("workclass").mean())   
        print(traindatafeatures.groupby("education-num").mean())   
        print(traindatafeatures.groupby("hours-per-week").mean()) 
        print(traindatafeatures['race'].value_counts())
        print(traindatafeatures.groupby("race").mean())   
        print(traindatafeatures.groupby("sex").mean())  
        print(traindatafeatures.groupby("occupation").mean())  
        print(traindatafeatures['occupation'].value_counts())
        
        #https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
        traindatafeatures=traindatafeatures.drop(columns =
                 ["fnlwgt","race", "sex",
                  "hours-per-week", "education-num","occupation","relationship"])
                                    
   
        #traindatafeatures=traindatafeatures.drop(columns = 
          #  ["workclass", "educatio'DataFrame' object has no attribute 'ravel'n","marital-status","occupation",
         #    "relationship","race","sex","native-country"])

        
        #print(traindatafeatures['education'].value_counts())
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Preschool',
                                                ' Some-school', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 1st-4th',
                                                ' Some-school', traindatafeatures['education'])       
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 5th-6th',
                                                ' Some-school', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 7th-8th',
                                                ' Some-school', traindatafeatures['education'])       
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 9th',
                                                ' Some-school', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 10th',
                                                ' Some-school', traindatafeatures['education'])       
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 11th',
                                                ' Some-school', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' 12th',
                                                ' Some-school', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Assoc-voc',
                                                ' Assoc-professional', traindatafeatures['education'])    
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Prof-school',
                                                ' Assoc-professional', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Assoc-acdm',
                                                ' Assoc-professional', traindatafeatures['education'])        
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Some-college',
                                                ' HS-grad or Some-college', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' HS-grad',
                                                ' HS-grad or Some-college', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Masters',
                                                ' Graduate-degree', traindatafeatures['education'])
        traindatafeatures['education']=np.where(traindatafeatures['education'] ==' Doctorate',
                                                ' Graduate-degree', traindatafeatures['education'])
        
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Divorced',
                                                ' Not Married', traindatafeatures['marital-status'])    
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Never-married',
                                                ' Not Married', traindatafeatures['marital-status'])    
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Separated',
                                                ' Not Married', traindatafeatures['marital-status'])    
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Widowed',
                                                ' Not Married', traindatafeatures['marital-status'])  
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Married-spouse-absent',
                                                ' Not Married', traindatafeatures['marital-status'])        
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Married-civ-spouse',
                                                ' Married', traindatafeatures['marital-status'])    
        traindatafeatures['marital-status']=np.where(traindatafeatures['marital-status'] ==' Married-AF-spouse',
                                                ' Married', traindatafeatures['marital-status'])  
       
      
        #Need to get capital gain and loss working! Make a new collumn?
        
        #traindatafeatures['capital-gain']=traindatafeatures['capital-gain'].astype(float)

        
        
        
        #print(traindatafeatures['capital-gain'].value_counts())
        
        #traindatafeatures['capital-gain']=np.where(traindatafeatures['capital-gain'].astype(float) < 149,
          #                                      ' Low', traindatafeatures['capital-gain'])
        
        #traindatafeatures['capital-gain']=np.where((traindatafeatures['capital-gain'].astype(float) > 149) &
        #         (traindatafeatures['capital-gain'].astype(float) <2077),' Mid-Low', traindatafeatures['capital-gain'])        
        #traindatafeatures['capital-gain']=np.where((traindatafeatures['capital-gain'] > 2077) &
           #      (traindatafeatures['capital-gain'] <4006),' Mid-High', traindatafeatures['capital-gain']) 
       
        #traindatafeatures['capital-gain']=np.where(int(traindatafeatures['capital-gain']) > 4006,
        #                                        ' High', traindatafeatures['capital-gain'])        

        """
        capital losses: 0, 53.142921, 124.0722255, 195.001530, and up
        
        %matplotlib inline
        pd.crosstab(traindatafeatures['capital-gain'],traindatafeatures['output']).plot(kind='bar')
        plt.title('Purchase Frequency for Job Title')
        plt.xlabel('Job')
        plt.ylabel('Frequency of Purchase')
        plt.savefig('purchase_fre_job')
        """
        
        #print(traindatafeatures['relationship'].unique())        
        #print(traindatafeatures['education'].value_counts())
        #print(traindatafeatures['capital-gain'].value_counts())
        #print(traincombined['output'].value_counts())
        #print(traindatafeatures.groupby("workclass").mean())
        #print(traindatafeatures.groupby('marital-status').mean())
        #print(traindatafeatures.groupby('age').mean())
        #print(traindatafeatures.groupby('native-country').mean())
        #print(traindatafeatures['native-country'].unique())   

In [7]:
data_a = DataA()
#https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a

                         age         fnlwgt  education-num  capital-gain  \
workclass                                                                  
 ?                 40.960240  188516.338235       9.260349    606.795752   
 Federal-gov       42.590625  185221.243750      10.973958    833.232292   
 Local-gov         41.751075  188639.712852      11.042045    880.202580   
 Never-worked      20.571429  225989.571429       7.428571      0.000000   
 Private           36.797585  192764.114734       9.879714    889.217792   
 Self-emp-inc      46.017025  175981.344086      11.137097   4875.693548   
 Self-emp-not-inc  44.969697  175608.641480      10.226289   1886.061787   
 State-gov         39.436055  184136.613251      11.375963    701.699538   
 Without-pay       47.785714  174267.500000       9.071429    487.857143   

                   capital-loss  hours-per-week    output  
workclass                                                  
 ?                    60.760349       31.91

In [ ]:
        
        #train_x,test_x,train_y,test_y= train_test_split(
         #      X,y, test_size=0.25, shuffle=True)